In [89]:
import pandas as pd
import yaml
import os
import plotly.express as px

In [90]:
max_dict = {
    'pl': 17722,
    'be': 25231, # TODO
    'fr': 14449,
    'en': 16621, # TODO
    'lv': 12520,
    'lt': 3642, # TODO
    'cs': 68494,
    'es': 17662, # TODO
    'zh': 3996,
    'slk': 8482,
    'slv': 10000, # TODO
    'uk': 5495,
    'ru': 10000, # TODO
    'de': 10000, # TODO
    'fi': 12215,
    'id': 4481,


}

In [91]:
slavic_langs = ("pl", "be", "cs", "slk", "slv", "uk", "ru")

In [92]:
# yaml_args = yaml.load(open(config))
results_dir = '/home/amysiak/thesis/multilingual-probing-visualization/probingOutputs'

In [93]:
df_data = {'dev': [], 'train': [], 'uuas': [], 'limit': []}

In [94]:
for subdir, dirs, files in os.walk(results_dir):
    # print(files)
    if len(files) < 5:
        continue

    for file in files:
        if file.endswith(".yaml"):
            config_args = yaml.load(open(os.path.join(subdir, file)))
            dset = config_args["dataset"]
            df_data['dev'].append(dset['keys']['dev'][0])
            df_data['train'].append(dset['keys']['train'][0])
            max_ = max_dict[dset['keys']['train'][0]]
            if 'limit' in dset.keys():
                df_data['limit'].append(min(max_, dset['limit']))
            else:
                df_data['limit'].append(max_)
            uuas_file = open(os.path.join(subdir, "dev.uuas"), 'r') 
            uuas_score = float(uuas_file.readlines() [0])
            uuas_file.close() 
            df_data['uuas'].append(uuas_score)

/tmp/ipykernel_3293874/4138612635.py:8: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [95]:
df = pd.DataFrame(df_data)

In [96]:
df["slavic"] = df.train.isin(slavic_langs)

In [97]:
df.tail()

,dev,train,uuas,limit,slavic
397,pl,uk,0.705509,2500,True
398,pl,zh,0.590002,2500,False
399,pl,zh,0.583088,2500,False
400,pl,zh,0.557319,2500,False
401,pl,zh,0.581085,2500,False


In [98]:
df = df[df.dev == 'pl']

In [99]:
pl_df = df[(df.dev == 'pl') & (df.train == 'pl')]

In [100]:
pl_df

,dev,train,uuas,limit,slavic
13,pl,pl,0.845122,17722,True
40,pl,pl,0.799403,5000,True
41,pl,pl,0.787199,5000,True
42,pl,pl,0.793849,5000,True
54,pl,pl,0.845689,17722,True
55,pl,pl,0.388763,100,True
56,pl,pl,0.836054,10000,True
57,pl,pl,0.799176,5000,True
90,pl,pl,0.575418,1000,True
91,pl,pl,0.758634,2500,True


In [101]:
fig = px.scatter(x=pl_df.limit, y=pl_df.uuas)
fig.show()

In [102]:
df_avg = df.groupby(["train", "limit"]).mean()
df_avg = df_avg.reset_index()

/tmp/ipykernel_3293874/3935998815.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [103]:
df_avg.head()

,train,limit,uuas,slavic
0,be,100,0.376049,1.0
1,be,1000,0.609452,1.0
2,be,2500,0.687618,1.0
3,be,3000,0.700314,1.0
4,be,5000,0.726457,1.0


In [104]:
df_avg["slavic"] = df_avg["slavic"].astype(str)

In [105]:
fig = px.scatter(data_frame=df_avg, x="limit", y="uuas", color="slavic", hover_data=["train"])
fig.show()

In [106]:
df = df.sort_values(by="limit")
df_avg = df_avg.sort_values(by="limit")

In [107]:
fig = px.line(data_frame=df_avg, x="limit", y="uuas", color="train", markers=True)
fig.show()